In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier,ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, train_test_split

properati = pd.read_csv('datos/properati_final.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

In [2]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['state_name']
le_barrio.fit(barrios)
properati['state_name'] = le_barrio.transform(barrios)

le_zona = preprocessing.LabelEncoder()
zona=properati['place_name']
le_zona.fit(zona)
properati['place_name'] = le_zona.transform(zona)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

# ET Clasificador

In [38]:
cant_buckets = 200

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 
properati.groupby('categories_by_price').count().sort_values("lat",ascending=True)

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,fecha,superficie_descubierta,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
177.0,20,20,20,20,20,15,15,20,20,20,...,20,3,13,1,20,20,20,20,15,20
198.0,22,22,22,22,22,19,19,22,22,22,...,22,1,15,4,22,22,22,22,13,22
193.0,21,21,21,21,21,19,19,21,21,21,...,21,0,11,2,21,21,21,21,11,21
175.0,28,28,28,28,28,20,20,28,28,28,...,28,1,22,5,28,28,28,28,23,28
173.0,29,29,29,29,29,22,22,29,29,29,...,29,1,18,2,29,29,29,29,24,29
195.0,29,29,29,29,29,26,26,29,29,29,...,29,2,20,4,29,29,29,29,15,29
189.0,35,35,35,35,35,29,29,35,35,35,...,35,2,25,11,35,35,35,35,23,35
187.0,39,39,39,39,39,30,30,39,39,39,...,39,0,29,6,39,39,39,39,26,39
199.0,37,37,37,37,37,30,30,37,37,37,...,37,2,32,7,37,37,37,37,19,37


In [ ]:
#preparo set de datos
X = zip()
y = properati['categories_by_price']

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

### Busco hiper-parametros con grid search

In [ ]:
%%notify

#preparo set de datos
X = zip()
y = properati['categories_by_price']

etc = ExtraTreesClassifier(n_jobs=-1)

bootstrap=[True,False]
max_features =["auto","sqrt","log2",None]

param_grid = {"bootstrap": bootstrap,"max_features":max_features}
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

grid_search = GridSearchCV(etc, param_grid=param_grid ,cv=5)
start = time()
grid_search.fit(X_train, X_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(grid_search.cv_results_['params'])))
print("")
score.report_single(grid_search.cv_results_)

# ET regresor

In [3]:
#preparo set de datos
X = zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['property_type'],properati['state_name'],properati['place_name'])
y = properati['price_aprox_usd']

### Random Search

In [5]:
%%notify
et = ExtraTreesRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["auto","log2",None]
n_estimators = np.arange(10,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(et, param_distributions=param_grid ,cv=5,n_iter=10) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 542.45 segundos para 10 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.677 (std: 0.143)
Promedio fit time: 4.130s
Hiper-parametros: {'n_estimators': 90, 'max_features': 'log2', 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.673 (std: 0.142)
Promedio fit time: 5.928s
Hiper-parametros: {'n_estimators': 120, 'max_features': 'log2', 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.668 (std: 0.140)
Promedio fit time: 9.032s
Hiper-parametros: {'n_estimators': 190, 'max_features': 'log2', 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.662 (std: 0.139)
Promedio fit time: 7.755s
Hiper-parametros: {'n_estimators': 150, 'max_features': 'log2', 'criterion': 'mse'}

Puesto: 5
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.

<IPython.core.display.Javascript object>

### Grid Search

In [ ]:
%%notify

et = ExtraTreesRegressor(n_jobs=-1)

criterion = ['mse']
max_features =['log2']
n_estimators = np.arange(30,70,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(et, param_grid=param_grid ,cv=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

In [ ]:
mejor_et = search.best_estimator_
print mejor_et.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

In [ ]:
search.score(X_test,y_test)

In [ ]:
errores = mejor_et.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print(errores)

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(errores, 100, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-1000, 1000) #para variar el "zoom a 0"
plt.yscale('log')
plt.show()